In [1]:
import pandas as pd
import numpy as np

In [2]:
edges = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
features = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_features.csv",header=None)
classes = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_classes.csv")

print(f'edges: {len(edges)}, features: {len(features)}, classes: {len(classes)}')

edges: 234355, features: 203769, classes: 203769


In [3]:
display(edges.head(5))

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [4]:
display(features.head(5))

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [5]:
features.iloc[:, :2].groupby(1).count()

,0
1,
1,7880
2,4544
3,6621
4,5693
5,6803
6,4328
7,6048
8,4457
9,4996


### Kroki czasowe
Kolumna 1 features pasuje opisem do time step z opisu danych z Kaggle

In [6]:
display(classes.head(5))

,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown


In [7]:
classes.groupby('class').count()

,txId
class,
1,4545
2,42019
unknown,157205


### Klasy
Na podstawie opisu z Kaggle i ilości klas:
- 1 - illicit
- 2 - licit
- unknown - unknown

### Podział dataframe'ów w zależności od klas

In [8]:
classes_illicit = classes[classes['class'] == '1']
classes_licit = classes[classes['class'] == '2']
classes_unknown = classes[classes['class'] == 'unknown']

display(classes_illicit.head(5), classes_licit.head(5), classes_unknown.head(5))

,txId,class
907,232629023,1
1361,230389796,1
2718,17387772,1
2815,232947878,1
3423,16754007,1


,txId,class
3,232438397,2
9,232029206,2
10,232344069,2
11,27553029,2
16,3881097,2


,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
4,230460314,unknown
5,230459870,unknown


In [9]:
classes_illicit['txId']

features.iloc[:, 0]

features.iloc[:, 0].isin(classes_illicit['txId'])

0         False
1         False
2         False
3         False
4         False
          ...  
203764    False
203765    False
203766     True
203767    False
203768    False
Name: 0, Length: 203769, dtype: bool

In [10]:
features_illicit = features[features.iloc[:, 0].isin(classes_illicit['txId'])]
features_licit = features[features.iloc[:, 0].isin(classes_licit['txId'])]
features_unknown = features[features.iloc[:, 0].isin(classes_unknown['txId'])]

print(f'features_illicit: {len(features_illicit)}, features_licit: {len(features_licit)}, features_unknown: {len(features_unknown)}')

features_illicit: 4545, features_licit: 42019, features_unknown: 157205


### Podział dataframe'ów w zależności od kroku czasowego

In [11]:
min_time_step = features[1].min()
max_time_step = features[1].max()

features_by_timestep = {}
edges_by_timestep = {}
classes_by_timestep = {}
for time_step in range(min_time_step, max_time_step):
    features_time_step = features[features.iloc[:, 1] == time_step]
    features_by_timestep[time_step] = features_time_step

    # According to description on Kaggle, there are no edges connecting transactions from different time steps.
    edges_time_step = edges[(edges.iloc[:, 0].isin(features_time_step[0])) | (edges.iloc[:, 0].isin(features_time_step[0]))]
    edges_by_timestep[time_step] = edges_time_step

    classes_time_step = classes[classes.iloc[:, 0].isin(features_time_step[0])]
    classes_by_timestep[time_step] = classes_time_step

In [12]:
features_time_step = features_by_timestep[1]
edges_time_step = edges_by_timestep[1]
classes_time_step = classes_by_timestep[1]

display(features_time_step, edges_time_step, classes_time_step)

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7875,230658152,1,-0.172879,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163539,...,1.268719,0.944323,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984
7876,54735200,1,-0.172980,-0.184668,-1.201369,-0.046932,-0.043875,-0.029140,-0.061584,-0.163645,...,-0.218398,-0.285627,-0.979074,-0.978556,0.018279,-0.068266,-0.084674,-0.054450,1.519700,1.521399
7877,230551382,1,-0.167408,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.157941,...,-0.472478,-0.474850,-0.979074,-0.978556,0.018279,-0.029817,0.008288,0.031697,-1.760926,-1.760984
7878,33654709,1,-0.169357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.159935,...,-0.569626,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792


,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206
...,...,...
9159,230437620,230439288
9160,203465969,5986851
9161,232051667,232051672
9162,232364495,34300577


,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown
...,...,...
7875,230658152,unknown
7876,54735200,2
7877,230551382,unknown
7878,33654709,unknown


### Macierz adjacencji

In [13]:
adjacency_matrix = pd.DataFrame(np.zeros((features_time_step.shape[0], features_time_step.shape[0])), index=features_time_step[0], columns=features_time_step[0])
display(adjacency_matrix)

,230425980,5530458,232022460,232438397,230460314,230459870,230333930,230595899,232013274,232029206,...,16642501,214640857,6255484,196468573,67286352,230658152,54735200,230551382,33654709,39251209
0,,,,,,,,,,,,,,,,,,,,,
230425980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5530458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232022460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232438397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
230460314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230658152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54735200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
230551382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
for i in range(edges_time_step.shape[0]):
    # A[j, i] != A[i, j] if A[i, j] == 1 because it's a Directed Acyclic Graph (DAG)
    adjacency_matrix.loc[edges_time_step.iloc[i]['txId1'], edges_time_step.iloc[i]['txId2']] = 1
display(adjacency_matrix)

,230425980,5530458,232022460,232438397,230460314,230459870,230333930,230595899,232013274,232029206,...,16642501,214640857,6255484,196468573,67286352,230658152,54735200,230551382,33654709,39251209
0,,,,,,,,,,,,,,,,,,,,,
230425980,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5530458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232022460,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232438397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
230460314,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230658152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54735200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
230551382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Drzewo transakcji wchodzących na podstawie macierzy adjacencji

In [15]:
root_transaction = edges_time_step.iloc[42]['txId1']
root_transaction

230585122

In [16]:
def get_transactions_in_with_adjacency_matrix(root_transaction, max_depth=None):
    transactions_in = [i for i in adjacency_matrix.index if adjacency_matrix.loc[i, root_transaction] == 1.0]
    transaction_trees_in = []
    if max_depth is None or max_depth > 0:
        for i in transactions_in:
            transaction_trees_in.append(get_transactions_in_with_adjacency_matrix(i, max_depth - 1 if max_depth is not None else None))
    else:
        transaction_trees_in = 'max_depth reached'

    transactions = {
        'txId': root_transaction,
        'transaction_trees_in': transaction_trees_in
    }
    return transactions


In [17]:
transactions_in = get_transactions_in_with_adjacency_matrix(edges_time_step.iloc[16]['txId1'])
transactions_in

{'txId': 233591710,
 'transaction_trees_in': [{'txId': 232007412,
   'transaction_trees_in': [{'txId': 232007419,
     'transaction_trees_in': [{'txId': 232686668,
       'transaction_trees_in': [{'txId': 232686665,
         'transaction_trees_in': [{'txId': 232655296,
           'transaction_trees_in': [{'txId': 230473474,
             'transaction_trees_in': []}]},
          {'txId': 232651633, 'transaction_trees_in': []},
          {'txId': 183781611,
           'transaction_trees_in': [{'txId': 232651633,
             'transaction_trees_in': []}]},
          {'txId': 232665894,
           'transaction_trees_in': [{'txId': 232655296,
             'transaction_trees_in': [{'txId': 230473474,
               'transaction_trees_in': []}]}]}]}]}]}]}]}

### Drzewo transakcji wychodzących na podstawie macierzy adjacencji

In [18]:
def get_transactions_out_with_adjacency_matrix(root_transaction, max_depth=None):
    transactions_out = [i for i in adjacency_matrix.columns if adjacency_matrix.loc[root_transaction, i] == 1.0]
    transaction_trees_out = []
    if max_depth is None or max_depth > 0:
        for i in transactions_out:
            transaction_trees_out.append(get_transactions_out_with_adjacency_matrix(i, max_depth - 1 if max_depth is not None else None))
    else:
        transaction_trees_out = 'max_depth reached'

    transactions = {
        'txId': root_transaction,
        'transaction_trees_out': transaction_trees_out
    }
    return transactions

In [19]:
transactions_out = get_transactions_out_with_adjacency_matrix(edges_time_step.iloc[16]['txId1'])
transactions_out

{'txId': 233591710,
 'transaction_trees_out': [{'txId': 234439913,
   'transaction_trees_out': [{'txId': 234441421,
     'transaction_trees_out': [{'txId': 233307261,
       'transaction_trees_out': [{'txId': 234472020,
         'transaction_trees_out': [{'txId': 234472023,
           'transaction_trees_out': [{'txId': 232413175,
             'transaction_trees_out': [{'txId': 232413172,
               'transaction_trees_out': [{'txId': 232413178,
                 'transaction_trees_out': [{'txId': 230556838,
                   'transaction_trees_out': []},
                  {'txId': 232419645,
                   'transaction_trees_out': [{'txId': 232419651,
                     'transaction_trees_out': [{'txId': 230556839,
                       'transaction_trees_out': []},
                      {'txId': 232419656,
                       'transaction_trees_out': [{'txId': 232419659,
                         'transaction_trees_out': [{'txId': 232420760,
                           'tra

### Mapa adjacencji

In [20]:
adjacency_map = {}
for index, row in edges_time_step.iterrows():
    if not row['txId1'] in adjacency_map:
        adjacency_map[row['txId1']] = []
    if not row['txId2'] in adjacency_map:
        adjacency_map[row['txId2']] = []
    adjacency_map[row['txId1']].append(row['txId2'])

adjacency_map

{230425980: [5530458],
 5530458: [232403360],
 232022460: [232438397, 232022462],
 232438397: [92491280],
 230460314: [230459870,
  230460307,
  230459688,
  230570333,
  230459930,
  3276131,
  230575202,
  230569936],
 230459870: [230457868, 230575202, 3276131, 3272533, 230459908, 230570333],
 230333930: [230595899, 232015639],
 230595899: [232675746],
 232013274: [232029206, 213003856],
 232029206: [5119722],
 232344069: [27553029, 234419685],
 27553029: [230411688],
 36411953: [230405052],
 230405052: [232383564],
 34194980: [5529846, 232896856],
 5529846: [232374910, 86495387],
 3881097: [232457116],
 232457116: [232457119],
 230409257: [32877982],
 32877982: [232899771],
 230351738: [195218118, 230457202],
 195218118: [232342220],
 88008478: [232012569],
 232012569: [230537142, 230518396],
 232412408: [232412405],
 232412405: [],
 232038018: [232470342, 230417053],
 232470342: [230516010],
 2925426: [230550393, 33972938],
 230550393: [],
 232051089: [232470704, 3875004],
 2324707

### Odwrócona mapa adjacencji

In [21]:
inversed_adjacency_map = {}
for index, row in edges_time_step.iterrows():
    if not row['txId1'] in inversed_adjacency_map:
        inversed_adjacency_map[row['txId1']] = []
    if not row['txId2'] in inversed_adjacency_map:
        inversed_adjacency_map[row['txId2']] = []
    inversed_adjacency_map[row['txId2']].append(row['txId1'])

inversed_adjacency_map

{230425980: [98374661],
 5530458: [230425980],
 232022460: [232000575],
 232438397: [232022460,
  232047899,
  3877118,
  230452718,
  230456717,
  230556037,
  230456719,
  232023529,
  230402893,
  230412047,
  208537161,
  230417569,
  3878718,
  230425284,
  230438854,
  232007342,
  230423848,
  230472241,
  232043907,
  230459331,
  16754007,
  230427509,
  232042947,
  230471948,
  230408405,
  230418063,
  230966618,
  230424541,
  18218289,
  230425547,
  7089693,
  230471939,
  230456734,
  230431165,
  71367715,
  230406812,
  231028597,
  231004407,
  230406773,
  230619786,
  230413257,
  3875725,
  230411283,
  230428237,
  3880228,
  232006620,
  230417077,
  230415432,
  230423018,
  3876512,
  231029351,
  230468036,
  230647320,
  14090056,
  231028826,
  231992576,
  230415434,
  230390558,
  230455503,
  230428187,
  230411209,
  3882627,
  204255781,
  230427132,
  3878606,
  230442070,
  232000563,
  230814630,
  230651381,
  230969378,
  232023175,
  230438845,
 

### Drzewo transakcji wchodzących na podstawie mapy adjacencji

In [22]:
def get_transactions_in_with_inversed_adjacency_map(root_transaction, max_depth=None):
    transactions_in = inversed_adjacency_map[root_transaction]
    transaction_trees_in = []
    if max_depth is None or max_depth > 0:
        for i in transactions_in:
            transaction_trees_in.append(get_transactions_in_with_inversed_adjacency_map(i, max_depth - 1 if max_depth is not None else None))
    else:
        transaction_trees_in = 'max_depth reached'

    transactions = {
        'txId': root_transaction,
        'transaction_trees_in': transaction_trees_in
    }
    return transactions


In [23]:
transactions_in = get_transactions_in_with_inversed_adjacency_map(edges_time_step.iloc[16]['txId1'])
transactions_in

{'txId': 233591710,
 'transaction_trees_in': [{'txId': 232007412,
   'transaction_trees_in': [{'txId': 232007419,
     'transaction_trees_in': [{'txId': 232686668,
       'transaction_trees_in': [{'txId': 232686665,
         'transaction_trees_in': [{'txId': 183781611,
           'transaction_trees_in': [{'txId': 232651633,
             'transaction_trees_in': []}]},
          {'txId': 232651633, 'transaction_trees_in': []},
          {'txId': 232665894,
           'transaction_trees_in': [{'txId': 232655296,
             'transaction_trees_in': [{'txId': 230473474,
               'transaction_trees_in': []}]}]},
          {'txId': 232655296,
           'transaction_trees_in': [{'txId': 230473474,
             'transaction_trees_in': []}]}]}]}]}]}]}

### Drzewo transakcji wychodzących na podstawie mapy adjacencji

In [24]:
def get_transactions_out_with_adjacency_map(root_transaction, max_depth=None):
    transactions_out = adjacency_map[root_transaction]
    transaction_trees_out = []
    if max_depth is None or max_depth > 0:
        for i in transactions_out:
            transaction_trees_out.append(get_transactions_out_with_adjacency_map(i, max_depth - 1 if max_depth is not None else None))
    else:
        transaction_trees_out = 'max_depth reached'

    transactions = {
        'txId': root_transaction,
        'transaction_trees_out': transaction_trees_out
    }
    return transactions

In [25]:
transactions_out = get_transactions_out_with_adjacency_map(edges_time_step.iloc[16]['txId1'])
transactions_out

{'txId': 233591710,
 'transaction_trees_out': [{'txId': 234439913,
   'transaction_trees_out': [{'txId': 234441421,
     'transaction_trees_out': [{'txId': 233307261,
       'transaction_trees_out': [{'txId': 234472020,
         'transaction_trees_out': [{'txId': 234472023,
           'transaction_trees_out': [{'txId': 232413175,
             'transaction_trees_out': [{'txId': 230556835,
               'transaction_trees_out': []},
              {'txId': 232413172,
               'transaction_trees_out': [{'txId': 19609789,
                 'transaction_trees_out': []},
                {'txId': 232413178,
                 'transaction_trees_out': [{'txId': 230556838,
                   'transaction_trees_out': []},
                  {'txId': 232419645,
                   'transaction_trees_out': [{'txId': 232419651,
                     'transaction_trees_out': [{'txId': 230556839,
                       'transaction_trees_out': []},
                      {'txId': 232419656,
          